In [1]:
#!pip install splink

In [2]:
import pandas as pd 
from splink.duckdb.duckdb_linker import DuckDBLinker
from splink.duckdb import duckdb_comparison_library as cl
import warnings
warnings.filterwarnings("ignore")
dup_df = pd.read_excel("Duplicates.xlsx")
main_df = pd.read_excel("Main Records.xlsx")
dup_df.head(2)

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue)
0,73c2ab6a,NaN,Steven,01/13/2008,NaN,NaN,NaN,Indian,NaN,Canada,NaN,NaN
1,488f1063,NaN,Dorothy,05/02/2012,NaN,NaN,Jewish,NaN,NaN,NaN,NaN,NaN


In [4]:
linker_eda = DuckDBLinker(dup_df)
#linker_eda.missingness_chart()

In [4]:
# Renaming columns for passing it to model
dup_df = dup_df.rename(columns={"Date of Birth": "dob"})
main_df = main_df.rename(columns={"Date of Birth": "dob"})

dup_df = dup_df.rename(columns={"First Name": "fn"})
main_df = main_df.rename(columns={"First Name": "fn"})

dup_df = dup_df.rename(columns={"Last Name": "ln"})
main_df = main_df.rename(columns={"Last Name": "ln"})

dup_df = dup_df.rename(columns={"Mother's Birth Last Name": "mbln"})
main_df = main_df.rename(columns={"Mother's Birth Last Name": "mbln"})

dup_df = dup_df.rename(columns={"Language (Mother Tongue)": "lmt"})
main_df = main_df.rename(columns={"Language (Mother Tongue)": "lmt"})

dup_df = dup_df.rename(columns={"ID": "unique_id"})
main_df = main_df.rename(columns={"ID": "unique_id"})

dup_df = dup_df.rename(columns={"Place of Birth": "pob"})
main_df = main_df.rename(columns={"Place of Birth": "pob"})

dup_df = dup_df.rename(columns={"Place of Birth": "pob"})
main_df = main_df.rename(columns={"Place of Birth": "pob"})

In [5]:
# Setting up the linker by defining blocking rules. Using levenshtein distance as distance algorithm
settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": ["l.dob = r.dob"],
    "comparisons": [
        cl.levenshtein_at_thresholds("fn"),
        cl.levenshtein_at_thresholds("ln"),
        cl.levenshtein_at_thresholds("mbln"),
        cl.exact_match("dob", term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("Citizenship"),
        cl.levenshtein_at_thresholds("lmt"),
        cl.levenshtein_at_thresholds("pob"),
        cl.levenshtein_at_thresholds("Ethnicity"),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

In [6]:
linker = DuckDBLinker([dup_df, main_df], settings ,input_table_aliases=["df_left", "df_right"])

# Specifying the rules
deterministic_rules = [
    "l.fn = r.fn and l.ln = r.ln and l.dob = r.dob",
    "l.dob = r.dob and levenshtein(r.fn, l.fn) <= 1 and levenshtein(r.ln, l.ln) <= 1",
    "l.dob = r.dob and levenshtein(r.fn, l.fn) <= 1 and levenshtein(r.ln, l.ln) <= 1",
    "l.dob = r.dob and l.fn = r.fn and l.mbln = r.ln",
    "l.dob = r.dob and l.fn = r.fn and l.mbln = r.ln",
    
    #"l.fn = r.fn and levenshtein(r.dob, l.dob) <= 1",
    #"l.ln = r.ln and levenshtein(r.dob, l.dob) <= 1"
    #"l.fn = r.fn and levenshtein(r.ln, l.ln) <= 2",
    #"l.fn = r.fn and levenshtein(r.mbln, l.mbln) <= 2"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


Probability two random records match is estimated to be  3.33e-05.
This means that amongst all possible pairwise record comparisons, one in 30,026.88 are expected to match.  With 202,810,140 total possible comparisons, we expect a total of around 6,754.29 matching pairs


In [7]:
linker.estimate_u_using_random_sampling(max_pairs=1e6)

----- Estimating u probabilities using random sampling -----
u probability not trained for Citizenship - Levenshtein <= 1 (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
u probability not trained for Citizenship - Levenshtein <= 2 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
u probability not trained for lmt - Levenshtein <= 1 (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
u probability not trained for lmt - Levenshtein <= 2 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
u probability not trained for pob - Levenshtein <= 1 (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
u probability not trained for Ethnicity - Levenshtein <= 1 (comparison vector value: 2). This usually m

In [8]:
session_dob = linker.estimate_parameters_using_expectation_maximisation("l.dob = r.dob")


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dob = r.dob

Parameter estimates will be made for the following comparison(s):
    - fn
    - ln
    - mbln
    - Citizenship
    - lmt
    - pob
    - Ethnicity

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob


Level Levenshtein <= 1 on comparison mbln not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison mbln not observed in dataset, unable to train m value

Level Levenshtein <= 1 on comparison Citizenship not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison Citizenship not observed in dataset, unable to train m value

Level Levenshtein <= 1 on comparison lmt not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison lmt not observed in dataset, unable to train m value

Level Levenshtein <= 1 on comparison pob not 

Iteration 7: Largest change in params was 0.000554 in the m_probability of ln, level `All other comparisons`

Level Levenshtein <= 1 on comparison mbln not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison mbln not observed in dataset, unable to train m value

Level Levenshtein <= 1 on comparison Citizenship not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison Citizenship not observed in dataset, unable to train m value

Level Levenshtein <= 1 on comparison lmt not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison lmt not observed in dataset, unable to train m value

Level Levenshtein <= 1 on comparison pob not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison pob not observed in dataset, unable to train m value

Level Levenshtein <= 1 on comparison Ethnicity not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison Ethnicity 

In [1]:
results = linker.predict(threshold_match_probability=0.8)

NameError: name 'linker' is not defined

In [10]:
results.as_pandas_dataframe()

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,fn_l,fn_r,gamma_fn,bf_fn,...,gamma_lmt,bf_lmt,pob_l,pob_r,gamma_pob,bf_pob,Ethnicity_l,Ethnicity_r,gamma_Ethnicity,bf_Ethnicity
0,7.211669,0.993299,df_left,df_right,6bc8df4b,b0b08352,Sally,Sally,3,225.377855,...,-1,1.000000,Canada,Canada,3,1.37931,NaN,NaN,-1,1.000000
1,8.946538,0.997977,df_left,df_right,b94bf3fd,36c72cab,Oliva,Oliva,3,225.377855,...,3,4.591025,NaN,NaN,-1,1.00000,NaN,NaN,-1,1.000000
2,17.177019,0.999993,df_left,df_right,b0a46c1a,15d64b9a,William,William,3,225.377855,...,-1,1.000000,Canada,Canada,3,1.37931,NaN,NaN,-1,1.000000
3,9.410485,0.998533,df_left,df_right,3212a968,33bcfbcd,Robert,Robert,3,225.377855,...,3,4.591025,Canada,Canada,3,1.37931,NaN,NaN,-1,1.000000
4,17.298035,0.999994,df_left,df_right,baf99f4a,c3e44793,Albert,Albert,3,225.377855,...,-1,1.000000,NaN,NaN,-1,1.00000,NaN,Alaska Native,-1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9673,9.148667,0.998241,df_left,df_right,b15fd5f8,98bf058e,Chi,Chi,3,225.377855,...,3,4.591025,NaN,NaN,-1,1.00000,Native Hawaiian,Native Hawaiian,3,8.052761
9674,27.033493,1.000000,df_left,df_right,1c5fcf8d,fe677b96,Mabel,Mabel,3,225.377855,...,-1,1.000000,NaN,NaN,-1,1.00000,Pacific Islander,Pacific Islander,3,8.052761
9675,16.713072,0.999991,df_left,df_right,7f93e1f6,ca94c645,Anthony,Anthony,3,225.377855,...,-1,1.000000,NaN,NaN,-1,1.00000,NaN,NaN,-1,1.000000
9676,16.298035,0.999988,df_left,df_right,e8cfb902,dcf1b191,Jason,Jason,3,225.377855,...,-1,1.000000,NaN,NaN,-1,1.00000,NaN,NaN,-1,1.000000


In [11]:
linker.match_weights_chart()

In [12]:
linker.m_u_parameters_chart()

In [13]:
linker.unlinkables_chart()

In [14]:
results = results.as_pandas_dataframe()
results.to_csv("linker_output.csv")

In [15]:
# Saving the model
settings = linker.save_settings_to_json("saved_model1.json", overwrite=True)